# Timeseries forecasting

Our goal is to be able to predict the number of rides in the local bike share program using forecasting methods that take historical data into account.

## Importing ride data

HealthyRide is the Pittsburgh bike share program. Data for all rides is made available at:

> https://healthyridepgh.com/data/

A file might have a first few lines like:

```
Trip id,Starttime,Stoptime,Bikeid,Tripduration,From station id,From station name,To station id,To station name,Usertype
15335599,1/1/2016 1:44,1/1/2016 2:01,70294,1068,1026,Penn Ave & S Whitfield St,1032,Walnut St & College St,
15335629,1/1/2016 2:39,1/1/2016 2:53,70360,892,1029,Alder St & S Highland Ave,1021,Taylor St & Liberty Ave,
```

The ride data from 2015 - 2017 are available in this project: 

```
datasets/
    HealthyRide Rentals 2015 Q2.csv
    HealthyRide Rentals 2015 Q3.csv
    HealthyRide Rentals 2015 Q4.csv
    HealthyRide Rentals 2016 Q1.csv
    HealthyRide Rentals 2016 Q2.csv
    HealthyRide Rentals 2016 Q3.csv
    HealthyRide Rentals 2016 Q4.csv
    HealthyRide Rentals 2017 Q1.csv
    HealthyRide Rentals 2017 Q2.csv
    HealthyRide Rentals 2017 Q3.csv
    HealthyRide Rentals 2017 Q4.csv
```

In [ ]:
import dask.dataframe as dd

rides = dd.read_csv('data/HealthyRide Rentals 201*.csv')

for c in 'Starttime','Stoptime':
    rides[c] = dd.to_datetime(rides[c], format='%m/%d/%Y %H:%M')
    

rides.head()

## Cleaning ride data and aggregate

There are times when cities have bike share services that a rider gives up on a ride, for whatever reason.  This would generally be reflected by them checking out a bike, but quickly realizing they cannot complete their ride, immediately returning it in the same place.  The data provides trip time in units of seconds.

Merely checking out and returning a bike to the same location does not mean the same thing.  For example, if a station is close to a resident's house, they might check a bike out, ride for an hour to complete errands, or just for leisure, and then come back to the same place.  We do not think of this as a "failed ride."

This cleaning step removes the data for "failed rides" from the DataFrame named `rides`.  A failed ride is defined as one:

1. From and to the same station
2. Duration of less than 120 seconds

In [ ]:
short = rides['Tripduration'] < 120
round_trip = rides['From station id'] == rides['To station id']
failed = short & round_trip
cleaned = rides[~failed]

Resample is a sophisticated groupby-like aggregation method in Pandas (and Dask). It will aggregate data according to a *frequency*. In this case the raw transactions are resampled to provide a count of the number of rides taken each day.

In [ ]:
daily = cleaned.set_index('Starttime')['Trip id'].resample('D').count()
daily.name = 'Count'

Here we can clearly see the trends in daily ridership.

In [ ]:
import hvplot.dask

daily.hvplot.line(width=800)

## Time series analysis: seasonal

[Statsmodels](https://www.statsmodels.org/stable/index.html) provides several useful methodology for timeseries analysis and modeling. We'll begin by attempting to decompose the average daily ride counts per week.

As should be obvious there is a strong seasonal variation yearly. Notice the small spread in the Trend values.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

weekly = daily.resample('W').mean().compute().asfreq('W-SUN')

result = seasonal_decompose(weekly)

In [ ]:
import pandas as pd
import hvplot.pandas

res = pd.DataFrame({
        'observed': result.observed,
        'seasonal': result.seasonal,
        'trend': result.trend,
        'residual': result.resid,
})

res.hvplot.line(subplots=True, height=200).cols(1)

## Stationarity

In order to fit a forecasting model the data must first be *stationary*, meaning that statistical properties like mean, variance, or autocorrelation are constant over time.

A standard test for stationarity is the [Augmented Dickey-Fuller](https://en.wikipedia.org/wiki/Augmented_Dickey%E2%80%93Fuller_test). The p-value is quite small so the weekly average series looks stationary so far.

In [ ]:
from statsmodels.tsa.stattools import adfuller

results = adfuller(weekly)
results[1]

## Model preparation

Analyzing the auto-correlation and partial auto-correlation functions will help us build the forecasting model later. 

There is a strong auto correlation every week.

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(weekly, lags=60);

If instead we plot the change in rides each week the correlation drops much faster.

In [ ]:
diff = weekly.diff().dropna()
plot_acf(diff, lags=60);
plot_pacf(diff, lags=60);

## ARIMA models

Our goal is to build a model that takes into account some amount of previous data. This is referred to as lags.

Three important parameters for the ARIMA model are

* **p** is the auto-regressive part of the model. It allows us to incorporate the effect of past values into our model. Intuitively, this would be similar to stating that it is likely to be warm tomorrow if it has been warm the past 3 days.
* **d** is the integrated part of the model. This includes terms in the model that incorporate the amount of differencing (i.e. the number of past time points to subtract from the current value) to apply to the time series. Intuitively, this would be similar to stating that it is likely to be same temperature tomorrow if the difference in temperature in the last three days has been very small.
* **q** is the moving average part of the model. This allows us to set the error of our model as a linear combination of the error values observed at previous time points in the past.

For convenience we'll use the [SARIMAX](https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html) method, but not include a seasonal component.

From the plots above we know that `d` should be 1, and both plots drop to zero quickly, so `p` and `q` can be set to 2.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_model import ARIMA

p = 2
d = 1
q = 2
order = (p, d, q)

arima = SARIMAX(weekly, order=order)
model = arima.fit(disp=0)

model.summary()

Plotting the original data and the fitted model shows close agreement with the forecasting model.

**Note**: the predictions here are made using the *real* historical data at each step. This is referred to as *one-step-ahead* prediction.

In [ ]:
predicted = model.predict()

weekly.hvplot.line(label='actual') * predicted.hvplot.line(label='one-step-ahead')

## Validation

The fit above looks good, but the *whole* data was used to train the model. Let's build a new model using only the first two years of data and use that to perform one-step-ahead predictions.

The `trained_model` has the parameters we wish to validate using unknown data.

In [ ]:
train = weekly.loc['2015':'2016']

arima = SARIMAX(train, order=order)
trained_model = arima.fit(disp=0)

To utilize the `.predict()` method we'll apply the pre-trained parameters to a model with *all* of the data.

In [ ]:
_arima = SARIMAX(weekly, order=order)
testing_model = _arima.filter(trained_model.params)

Again, the one-step-ahead predictions are made using the actual previous values, but the model was not fit with that data.

In [ ]:
predicted = testing_model.get_prediction(full_results=True).summary_frame()

In [ ]:
actual    = weekly.hvplot.line(label='actual')
one_ahead = predicted.loc['2017':, 'mean'].hvplot.line(label='one-step-ahead')
ci95      = predicted.loc['2017':].hvplot.area(y='mean_ci_lower', y2='mean_ci_upper',
                                               alpha=0.2, label='95% confidence')

(actual * one_ahead * ci95).opts(width=800, legend_position='top_left')

## Save the model

In [ ]:
testing_model.save('bike_forecasts.pkl')

In the next notebook the model is loaded and forecasting function is defined. Click the blue link to open the notebook.

<a href='load_model.ipynb' class='btn btn-primary btn-lg'>Load model and predict</a>

----

<font color='grey'><i>Copyright Anaconda 2012-2019 All Rights Reserved.</i></font>